In [ ]:
# conda list --export > requirements_conda.txt

In [13]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from tools.dset_getters import dummy_dset, cvsi_dset, me_dset
from tools.utils import coll
from tools.basic_model import NeuralNetwork

In [14]:

# a dummy model with an adjustable number of classes
class NeuralNetwork(nn.Module):
    def __init__(self, number_of_classes):
        super(NeuralNetwork, self).__init__()
        # self.flatten = nn.Flatten()
        self.lin1 = nn.Linear(4707,512)
        self.relu = nn.ReLU()
        self.lin2 = nn.Linear(512, number_of_classes)
        self.softmax = nn.Softmax(dim=0)
        self.loss = torch.nn.functional.binary_cross_entropy_with_logits

    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.softmax(x)

        return x
        
    def train_loop(self, dataloader, optimizer):
        for x, y in dataloader:
            pred = model(x)
            loss = self.loss(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
    def train(self, dataloader, epochs):
        learning_rate = 1e-3
        optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
        for t in range(epochs):
            print(f"Epoch {t}")
            model.train_loop(dataloader, optimizer)

    def test(self, dataloader):
        size = dataloader.dataset.__len__()
        num_batches = len(dataloader)
        test_loss, correct = 0, 0

        with torch.no_grad():
            for X, y in dataloader:
                pred = model(X)
                test_loss += self.loss(pred, y).item()
                correct += (torch.round(pred) == y).sum().item()

        print('test_loss ', test_loss)
        print('num_batches', num_batches)
        print('correct', correct)
        print('size', size)

        test_loss /= num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [20]:


device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

get_dataset_fn = cvsi_dset
train_dset = get_dataset_fn('train', force_download=False)
train_loader = DataLoader(train_dset, batch_size=32, shuffle=True, collate_fn=coll)


Using cuda device


fasta_cache/Homo_sapiens.GRCh38.cdna.all.fa.gz: 0.00B [00:00, ?B/s]

fasta_cache/Homo_sapiens.GRCh38.dna.toplevel.fa.gz: 0.00B [00:00, ?B/s]

  0%|          | 0/190000 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

In [21]:
max_len = -1
for i in range(len(train_dset)):
    current = len(train_dset[i][0])
    if max_len < current: max_len = current
    
print(max_len)

200


In [22]:
# print(next(iter(train_loader))[0][0])
# print(next(iter(train_loader))[1][0])

In [23]:
model = NeuralNetwork(number_of_classes=1).cuda()
model.train(train_loader, epochs=5)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [24]:
test_dset = get_dataset_fn('test', force_download=False)
test_loader = DataLoader(test_dset, batch_size=32, shuffle=True, collate_fn=coll)
model.test(test_loader)

Reference fasta_cache/Homo_sapiens.GRCh38.cdna.all.fa.gz already exists. Skipping.
Reference fasta_cache/Homo_sapiens.GRCh38.dna.toplevel.fa.gz already exists. Skipping.


  0%|          | 0/190000 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

test_loss  541.8566455841064
num_batches 782
correct 12500
size 25000
Test Error: 
 Accuracy: 50.0%, Avg loss: 0.692911 

